<a href="https://colab.research.google.com/github/ilyanovak/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Copy_of_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
# Split dataset into training and validation datasets
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, 
                              train_size=0.8, 
                              test_size=0.2, 
                              stratify=train['status_group'], 
                              random_state=42) 

In [0]:
import numpy as np

def wrangle (X):
  
  # Prevent SettingWithCopyWarning
  X = X.copy()
  
  # Prevent null island
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # Replace zeros with nans
  # Add column describing missing values
  cols_with_zero = ['population', 
                    'construction_year',
                    'region_code', 
                    'district_code', 
                    'longitude', 
                    'latitude',
                    'gps_height']
  for col in cols_with_zero:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_Missing'] = X[col].isna()

  # Drop columns with values which never vary
  unusuable_variance = ['id']
  X = X.drop(columns=unusuable_variance)

  # Convert date_recorded to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  # Extract datetime components from date_recorded
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day

  # Add column, how many years from construction to present
  X['age'] = X['year_recorded'] - X['construction_year']

  # Add Missing columns for columsnw with many nans
  cols_with_nans = ['funder', 'installer', 'subvillage', 'public_meeting', 
                    'scheme_management', 'scheme_name', 'permit', 'age']
  for col in cols_with_nans:
    X[col+'_Missing'] = X[col].isna()

  # Convert status_group to integer dtype
  if ('status_group' in X.columns):
    le = LabelEncoder()
    X['status_group'] = le.fit_transform(X['status_group'])

  # Return wrangled dataframe
  return X

train_wrangle = wrangle(train)
val_wrangle = wrangle(val)
test_wrangle = wrangle(test)

In [0]:
# Target colunns
target = 'status_group'

# Features columns
train_features = train_wrangle

# Numeric features columns
numeric_features = train_wrangle.select_dtypes(include='number').columns.tolist()

# Obtain cardinality of categorical features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Categorical features with cardinality less than 150
categorical_features = cardinality[cardinality < 50].index.tolist()

# Combine to lists of features
features = numeric_features + categorical_features

# Drop status_group
features.remove('status_group')

In [0]:
# Arrange data into X features and Y label
X_train = train_wrangle[features]
y_train = train_wrangle[target]
X_val = val_wrangle[features]
y_val = val_wrangle[target]
X_test = test_wrangle[features]

In [0]:
param_distributions = {
  'simpleimputer__strategy': ['mean', 'median'],
  # 'selectkbest__k': range(1, X_train.shape[1]+1), 
  'randomforestclassifier__n_estimators': randint(50, 500), 
  'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
  'randomforestclassifier__max_features': uniform(0, 1),
  'randomforestclassifier__class_weight': ['balanced', 'balanced_subsample', None], 
  'randomforestclassifier__min_samples_leaf': randint(1, 1000),
}

In [0]:
pipeline3 = make_pipeline(
  ce.one_hot.OrdinalEncoder(),
  SimpleImputer(),
  # SelectKBest(),
  RandomForestClassifier(n_jobs=-1)
)

search3 = RandomizedSearchCV(
    pipeline3, 
    param_distributions=param_distributions, 
    n_iter=20, 
    cv=5, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search3.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 30.8min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 36.0min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout o

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [0]:
pipeline3 = search3.best_estimator_

In [0]:
y_pred = pipeline3.predict(X_val)

y_pred = pd.Series(y_pred)
y_pred = le.inverse_transform(y_pred)
y_val = le.inverse_transform(y_val)

accuracy_score(y_pred, y_val)

0.7546296296296297